In [3]:
import tensorflow as tf

In [4]:
tf.__version__

'2.0.0'

In [5]:
from tensorflow.keras import layers as kl
from tensorflow.keras import backend as K

In [6]:
issubclass(kl.Layer, tf.Module)

True

We want to have all layer-variables in correct name-spaces also in cases where the standard keras Layer `call`-`build` mechanism is not applicable.
# Simple way to organize it

In [31]:
class MySublayer(kl.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        
        with tf.name_scope(self.name_scope()):
            self.b = tf.Variable(0., name="b", dtype=K.floatx())
            
    def my_call(self, x):
        return x + self.b

In [38]:
class MyLayer(kl.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        
        with tf.name_scope(self.name_scope()):
            self.sublayer = MySublayer()
            self.a = tf.Variable(1., name="a", dtype=K.floatx())
    
    def my_call(self, x):
        with tf.name_scope(self.name_scope()):
            return self.sublayer.my_call(self.a * x)

In [39]:
l = MyLayer()
l.weights

[<tf.Variable 'my_layer_12/a:0' shape=() dtype=float32, numpy=1.0>,
 <tf.Variable 'my_layer_12/my_sublayer_4/b:0' shape=() dtype=float32, numpy=0.0>]

In [40]:
l.sublayer.weights

[<tf.Variable 'my_layer_12/my_sublayer_4/b:0' shape=() dtype=float32, numpy=0.0>]

In [42]:
x = tf.Variable(1., name="x")

In [43]:
l.my_call(x)

<tf.Tensor: id=115, shape=(), dtype=float32, numpy=1.0>

# Better way to organize it
The above will not work correctly if the sublayer is created outside of the super-layer. Here is how to deal with it.

In [34]:
class MySublayer(kl.Layer):    
    def build(self, input_shapes):
        self.b = tf.Variable(2., name="b", dtype=K.floatx())
            
    def my_call(self, x):
        with tf.name_scope(self.name_scope()):
            if not self.built:
                self.build(tf.shape(x))
                self.built = True

            return x + self.b

In [35]:
class MyLayer(kl.Layer):
    def __init__(self, sublayer: kl.Layer, **kwargs):
        super().__init__(**kwargs)
        self.sublayer = sublayer
        
    def build(self, input_shapes):
        self.a = tf.Variable(-1., name="a", dtype=K.floatx())
    
    def my_call(self, x):
        with tf.name_scope(self.name_scope()):
            if not self.built:
                self.build(tf.shape(x))
                self.built = True
            return self.sublayer.my_call(self.a * x)

In [36]:
l = MyLayer(sublayer=MySublayer())
l.weights

[]

In [37]:
l.my_call(2.)

<tf.Tensor: id=96, shape=(), dtype=float32, numpy=0.0>

In [38]:
l.weights

[<tf.Variable 'my_layer_4/a:0' shape=() dtype=float32, numpy=-1.0>,
 <tf.Variable 'my_layer_4/my_sublayer_4/b:0' shape=() dtype=float32, numpy=2.0>]

### Try to save it and inspect the `hdf5` file structure

In [1]:
from tensorflow.keras import Model

In [51]:
class MyModel(Model):
    def __init__(self, layer, **kwargs):
        super().__init__(**kwargs)
        self.layer = layer
        
    def call(self, x):
        return self.layer.my_call(x)

In [52]:
model = MyModel(layer=MyLayer(sublayer=MySublayer()))

In [54]:
model(0.)

<tf.Tensor: id=125, shape=(), dtype=float32, numpy=2.0>

In [55]:
model.weights

[<tf.Variable 'my_model_4/my_layer_7/a:0' shape=() dtype=float32, numpy=-1.0>,
 <tf.Variable 'my_model_4/my_layer_7/my_sublayer_7/b:0' shape=() dtype=float32, numpy=2.0>]

In [56]:
model.save_weights("d:/to_delete_weights.h5")